In [ ]:
%qtconsole

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import levitate
from importlib import reload


In [ ]:
transducer = levitate.models.CircularRing(freq=40e3, effective_radius=2.9e-3)
array = levitate.models.TransducerArray(transducer_model=transducer)

In [ ]:
loc = np.array([0, 0, 0.15])
optim = levitate.optimization.Optimizer(array)
gorkov = levitate.optimization.gorkov_laplacian(array=array, location=loc, weights=(1e-3, -1, -1, -100))
optim.objectives.append(gorkov)

In [ ]:
starting_phase = np.random.uniform(-np.pi, np.pi, array.num_transducers)
array.phases = starting_phase
array.amplitudes = np.ones(array.num_transducers)

In [ ]:
optim.basinhopping = True
optim.variable_amplitudes = True
# %timeit optim.func_and_jac(array.phases)
# %snakeviz optim()
# %lprun -f optim.func_and_jac optim()
optim()
array.phases = np.mod(optim.phases + np.pi,2*np.pi)-np.pi
array.amplitudes = optim.amplitudes

In [ ]:
plot_loc = loc
p_scale = lambda p: 20*np.log10(np.abs(p)) + 94
p_range = (140, 160)
resolution = array.wavelength/15
xz_mesh = np.meshgrid(np.arange(-0.1, 0.1, resolution), np.arange(0, 0.3, resolution), indexing='ij')
pressures_xz = np.squeeze(array.calculate_pressure((xz_mesh[0], plot_loc[1]*np.ones_like(xz_mesh[0]), xz_mesh[1])))

fig = plt.figure()
plt.pcolormesh(xz_mesh[0], xz_mesh[1], p_scale(pressures_xz))
plt.xlabel('x')
plt.ylabel('z')
plt.clim(p_range)
plt.colorbar()
plt.plot(loc[0], loc[2], color='red', marker='o', markerfacecolor='red')
fig.show()

In [ ]:
x_range = np.arange(-5e-3, 5e-3, resolution) + loc[0]
x_force = []
for x in x_range:
    force_calc = levitate.optimization.gorkov_divergence(array, (x, loc[1], loc[2]))
    x_force.append(force_calc(optim.result.x))
x_force = -np.array(x_force)
plt.plot(x_range*1e3, x_force)
plt.title('Forces moving along x-axis')
plt.xlabel('x in mm')
plt.ylabel('Force in N')
plt.legend(['F_x', 'F_y', 'F_z'])
plt.show()

y_range = np.arange(-5e-3, 5e-3, resolution) + loc[1]
y_force = []
for y in y_range:
    force_calc = levitate.optimization.gorkov_divergence(array, (loc[0], y, loc[2]))
    y_force.append(force_calc(optim.result.x))
y_force = -np.array(y_force)
plt.plot(y_range*1e3, y_force)
plt.title('Forces moving along y-axis')
plt.xlabel('y in mm')
plt.ylabel('Force in N')
plt.legend(['F_x', 'F_y', 'F_z'])
plt.show()

z_range = np.arange(-5e-3, 5e-3, resolution) + loc[2]
z_force = []
for z in z_range:
    force_calc = levitate.optimization.gorkov_divergence(array, (loc[0], loc[1], z))
    z_force.append(force_calc(optim.result.x))
z_force = -np.array(z_force)
plt.plot(z_range*1e3, z_force)
plt.title('Forces moving along z-axis')
plt.xlabel('z in mm')
plt.ylabel('Force in N')
plt.legend(['F_x', 'F_y', 'F_z'])
plt.show()